In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import StamenTonerTilesAccess as STT
from sklearn.neighbors import KernelDensity
from sklearn.grid_search import GridSearchCV
import requests
import json
import StringIO

In [14]:
# DON'T RUN THIS CELL UNLESS THE CRIME_DATA FILE IS LOST

# This is the URL for the Philly PD crime incidents 2006-present API, and the requested data is given as a csv
url = 'https://data.phila.gov/resource/sspu-uyfa.csv?'

# This is my identifier 
token = '$$app_token=r3XjAbgHWtqqrmZCcdIpEd97O'

# By default, a request only gets me 1000 points, but there are over 2 million. I set limit to 3 million
limit = '&$limit=3000000'

# The API allows me to select columns a priori
cols= '&$select='

# These are the columns I want
col1 = 'dispatch_date AS Date'
col2 = ',dispatch_time AS Time'
col3 = ',ucr_general AS CrimeCodeNo'
col4 = ',text_general_code AS CrimeCodeText'
col5 = ',point_x AS lon'
col6 = ',point_y AS lat'

try:
    # Append the url and request the data
    response = requests.get(url+token+limit+cols+col1+col2+col3+col4+col5+col6)
    
    # Make the response readable
    csvdata = StringIO.StringIO(response.content)
    
    # Store it in a dataframe
    df = pd.read_csv(csvdata)
    
    # Pickle the dataframe so we don't have to keep requesting each time we want to use the data,
    # we can just load using pd.read_pickle('crime_data')
    df.to_pickle('crime_data')

except requests.exceptions.RequestException as e:
    print e
    sys.exit(1)


In [15]:
# Read data from pickle if we are starting a new session
df = pd.read_pickle('crime_data')

In [7]:
df.head()

,CrimeCodeNo,CrimeCodeText,Date,Time,lat,lon
0,800,Other Assaults,2009-10-02,14:24:00,NaN,NaN
1,2600,All Other Offenses,2009-05-10,00:55:00,NaN,NaN
2,800,Other Assaults,2009-08-07,15:40:00,NaN,NaN
3,1500,Weapon Violations,2009-07-19,01:09:00,40.036389,-75.130477
4,2600,All Other Offenses,2009-06-25,00:14:00,39.969532,-75.166350


In [8]:
# First let's drop the data points which have no latitude or longitude

df = df.dropna()
df.head()

,CrimeCodeNo,CrimeCodeText,Date,Time,lat,lon
3,1500,Weapon Violations,2009-07-19,01:09:00,40.036389,-75.130477
4,2600,All Other Offenses,2009-06-25,00:14:00,39.969532,-75.166350
5,600,Thefts,2015-04-25,12:50:00,39.940070,-75.166412
6,800,Other Assaults,2009-02-10,14:33:00,39.979586,-75.171149
12,600,Thefts,2015-10-06,18:18:00,39.976134,-75.160229


In [9]:
# Next, let's identify the crime code numbers corresponding to the different offenses

df.groupby(['CrimeCodeNo','CrimeCodeText']).count()

Date    Time     lat  \
CrimeCodeNo CrimeCodeText                                                     
100         Homicide - Criminal                        3188    3188    3188   
            Homicide - Gross Negligence                  11      11      11   
            Homicide - Justifiable                       41      41      41   
200         Rape                                      10860   10860   10860   
300         Robbery Firearm                           37898   37898   37898   
            Robbery No Firearm                        48507   48507   48507   
400         Aggravated Assault Firearm                25895   25895   25895   
            Aggravated Assault No Firearm             64270   64270   64270   
500         Burglary Non-Residential                  22100   22100   22100   
            Burglary Residential                      89670   89670   89670   
600         Theft from Vehicle                       158636  158636  158636   
            Thefts                                   236874  236874  236874   
700         Motor Vehicle Theft                       44462   44462   44462   
            Recovered Stolen Motor Vehicle            88130   88130   88130   
800         Other Assaults                           257464  257464  257464   
900         Arson                                      5309    5309    5309   
1000        Forgery and Counterfeiting                 4651    4651    4651   
1100        Fraud                                    104527  104527  104527   
1200        Embezzlement                               4334    4334    4334   
1300        Receiving Stolen Property                   738     738     738   
1400        Vandalism/Criminal Mischief              187252  187252  187252   
1500        Weapon Violations                         17460   17460   17460   
1600        Prostitution and Commercialized Vice      12094   12094   12094   
1700        Other Sex Offenses (Not Commercialized)   14520   14520   14520   
1800        Narcotic / Drug Law Violations           130502  130502  130502   
1900        Gambling Violations                         883     883     883   
2000        Offenses Against Family and Children       1694    1694    1694   
2100        DRIVING UNDER THE INFLUENCE               50088   50088   50088   
2200        Liquor Law Violations                      5340    5340    5340   
2300        Public Drunkenness                         4420    4420    4420   
2400        Disorderly Conduct                        38997   38997   38997   
2500        Vagrancy/Loitering                         6734    6734    6734   
2600        All Other Offenses                       412901  412901  412901   

                                                        lon  
CrimeCodeNo CrimeCodeText                                    
100         Homicide - Criminal                        3188  
            Homicide - Gross Negligence                  11  
            Homicide - Justifiable                       41  
200         Rape                                      10860  
300         Robbery Firearm                           37898  
            Robbery No Firearm                        48507  
400         Aggravated Assault Firearm                25895  
            Aggravated Assault No Firearm             64270  
500         Burglary Non-Residential                  22100  
            Burglary Residential                      89670  
600         Theft from Vehicle                       158636  
            Thefts                                   236874  
700         Motor Vehicle Theft                       44462  
            Recovered Stolen Motor Vehicle            88130  
800         Other Assaults                           257464  
900         Arson                                      5309  
1000        Forgery and Counterfeiting                 4651  
1100        Fraud                                    104527  
1200        Embezzlement                         

In [10]:
# We see that there are actually a large number of thefts. For now, let us focus on aggravated crimes like
# homicide, robbery, rape, assualt, burglary. These have crime codes 100-500 and 800
# Later, we can study motor vehicle thefts as well

# For now, we're also not going to care about time so we won't deal with this now. Later on, we can see how 
# crime changes based on times of the day

crime_df = df[(df.CrimeCodeNo<500) | (df.CrimeCodeNo==800)].drop('Time',axis=1)

# We can reset the index. This way, we won't run into problems later on when getting the year

crime_df = crime_df.reset_index()
crime_df.head()

,index,CrimeCodeNo,CrimeCodeText,Date,lat,lon
0,6,800,Other Assaults,2009-02-10,39.979586,-75.171149
1,28,800,Other Assaults,2015-08-23,39.995626,-75.125869
2,35,300,Robbery No Firearm,2015-11-13,40.020219,-75.077126
3,36,800,Other Assaults,2015-10-29,39.968878,-75.236743
4,52,300,Robbery Firearm,2015-11-05,39.986415,-75.169577


In [11]:
import math

# For now, we are only interested in the year the crime took place
test = [int(date[0:4]) for date in crime_df.Date]
crime_df['Year'] = pd.Series([int(date[0:4]) for date in crime_df.Date])
crime_loc_df = crime_df[['lat','lon','Year']]
test = crime_loc_df[crime_loc_df.Year==2006]

In [12]:
# This is for plotting the crime on a map

# Find Max and Min Longitude value

latmax = crime_df['lat'].max()   #Northmost point : 40.137445
latmin = crime_df['lat'].min()   #Southmost point : 39.874131
lonmax = crime_df['lon'].max()  #Eastmost point : -74.957602
lonmin = crime_df['lon'].min()   #Westmost point :-75.275694
lonlat = zip(crime_df.lon.values, crime_df.lat.values)
lat_deg = latmin
lon_deg = lonmin
delta_lat = latmax-latmin
delta_long = lonmax-lonmin
zoom = 14

#lat_deg, lon_deg, delta_lat,  delta_long, zoom = lat_deg-delta_lat/2, lon_deg-delta_long/2, delta_lat,  delta_long, 11
#lat_deg, lon_deg, delta_lat,  delta_long, zoom = 45.720-0.04/2, 4.210-0.08/2, 0.04,  0.08, 14
a, bbox = STT.getImageCluster(lat_deg, lon_deg, delta_lat,  delta_long, zoom)

Opening: http://tile.stamen.com/toner/14/4766/6194.png
Opening: http://tile.stamen.com/toner/14/4766/6195.png
Opening: http://tile.stamen.com/toner/14/4766/6196.png
Opening: http://tile.stamen.com/toner/14/4766/6197.png
Opening: http://tile.stamen.com/toner/14/4766/6198.png
Opening: http://tile.stamen.com/toner/14/4766/6199.png
Opening: http://tile.stamen.com/toner/14/4766/6200.png
Opening: http://tile.stamen.com/toner/14/4766/6201.png
Opening: http://tile.stamen.com/toner/14/4766/6202.png
Opening: http://tile.stamen.com/toner/14/4766/6203.png
Opening: http://tile.stamen.com/toner/14/4766/6204.png
Opening: http://tile.stamen.com/toner/14/4766/6205.png
Opening: http://tile.stamen.com/toner/14/4766/6206.png
Opening: http://tile.stamen.com/toner/14/4766/6207.png
Opening: http://tile.stamen.com/toner/14/4766/6208.png
Opening: http://tile.stamen.com/toner/14/4766/6209.png
Opening: http://tile.stamen.com/toner/14/4766/6210.png
Opening: http://tile.stamen.com/toner/14/4767/6194.png
Opening: h

In [13]:
xmin = bbox[0]
ymin = bbox[1]
xmax = bbox[2]
ymax = bbox[3]
x = np.arange(xmin, xmax,0.003)
y = np.arange(ymin, ymax, 0.003)
X,Y = np.meshgrid(x,y)
xy = np.vstack([Y.ravel(),X.ravel()]).T

fig = plt.figure(figsize=(10, 10)) 
ax = plt.subplot(111)
m = Basemap(
    llcrnrlon=bbox[0], llcrnrlat=bbox[1],
    urcrnrlon=bbox[2], urcrnrlat=bbox[3],
    projection='merc', ax=ax
)
m.imshow(a, interpolation='lanczos', origin='upper')
plt.show()

fig, ax = plt.subplots()
ax = sns.kdeplot(test.lon.values, test.lat.values, shade=True) 


# fig, ax = plt.subplots()
# for bandwidth in [0.1, 0.3, 1.0]:
#     ax.plot(Z,
#             label='bw={0}'.format(bandwidth), linewidth=3, alpha=0.5)